In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset, random_split

import tqdm
import re

In [2]:
path = '/kaggle/input/sarcasm-detection/sarcasm_detection.csv'

In [3]:
data = pd.read_csv(path)
data = data.drop_duplicates()
data = data.loc[:, ['is_sarcastic', 'headline']]

In [4]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
batch_size = 32


In [5]:
import torchtext
import re
import string


class SarcasmDataset(Dataset):
    def __init__(self, data,
                 vectorizer = torchtext.vocab.GloVe(name='6B'),
                 max_sequence_length: int=15,
                 transform = None):
        super().__init__()
        
        self.data = data
        self.transform = transform
        self.vectorizer = vectorizer
        self.max_sequence_length = max_sequence_length
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data.iloc[idx]

        text = item.headline
        target = item.is_sarcastic
        
        text = text.lower().strip()
        text_words = re.sub('\?|\!|\.|\,|\;|\'|\"', '', text).split(' ')
        
        zero_vector = torch.zeros_like(self.vectorizer['i'])
        vectors = [self.vectorizer[x] if x in self.vectorizer.stoi else zero_vector for x in text.split()]
#Если разница между заданной и текущей длиной меньше нуля,то ми не добавляем нулевіе вектора
#Добавляем их что наполнить наш лист 
        for _ in range(max(0, self.max_sequence_length - len(vectors))):
            vectors.append(zero_vector)
            
#Может бить такое,что некоторие предложенія меньше значенія,а некоторие больше
        vectors = vectors[:self.max_sequence_length]

#Переводім в торчевскій
        vectors = torch.stack(vectors)
    
        if self.transform:
            text = self.transform(text)
#Возвращаем лібрарі 
        return {'target': target,
                'vectors': vectors }
    

dataset = SarcasmDataset(data)
trainset, validset = random_split(dataset, [0.7, 0.3])
train_dataloader = DataLoader(trainset, shuffle=True, batch_size=batch_size)
valid_dataloader = DataLoader(validset, shuffle=True, batch_size=batch_size)


.vector_cache/glove.6B.zip: 862MB [02:39, 5.42MB/s]                              
100%|█████████▉| 399999/400000 [01:08<00:00, 5876.26it/s]


In [6]:
import torch
import torch.nn as nn

class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()
        
        self.lstm = nn.Sequential(
        nn.LSTM(input_size = 300, hidden_size = 15, num_layers = 1)
        )
        #proj_size = 0, то каждий виход імеет размер hidden_size
        #Конвертуємо,щоб не було помикли несумісних до множення матриць
        self.double()
        
        self.flatten = nn.Flatten()
        
        self.lin1 = nn.Sequential(
            nn.Dropout(0.1),
            nn.Linear(225, 1))

    def forward(self, x):
        out = x
        
        out, _ = self.lstm(out)
        out = out.to(torch.float32)
        out = self.flatten(out)
        out = self.lin1(out)

        
        return out
    


In [ ]:
from torchmetrics.classification import BinaryAccuracy

model = LSTM().to(dtype=torch.float32, device='cuda')
loss_fn = nn.BCEWithLogitsLoss() 
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5,weight_decay=1e-3) 
metrics = BinaryAccuracy().to(device)

val_loss_g = [1] #Список середніх для кожної ітерації втрат

i = 1
while True:
    print(f"Epoch {i}")
        # Train loop
    train_tqdm = tqdm.tqdm(train_dataloader)
    model.train()
    for batch in train_tqdm:
        sentence, cls = batch['vectors'], batch['target'] #Разделяем веткора і цели
        sentence, cls = sentence.to(device), cls.to(device) #Переводім в відеокарту

        optimizer.zero_grad()  #Обнуляем везде градиенти
        model.zero_grad()

        output = model(sentence).squeeze(1) #Робимо предікт та зменшуємо на одну розмірність 

        L = loss_fn(output, cls.float())  #Рахуємо втрати
        L.backward()   #Градієнт втрат


        optimizer.step()   #Оновлюємо ваги в нейронці
        train_tqdm.set_description(f"Train loss: {L.item()}") #Виводимо в термінал інформацію про втрати
        train_tqdm.refresh() #Оновлюємо для кожного батчу термінал 
        # Validation loop
    model.eval() #Модель тепер для передбачення
    
    valid_tqdm = tqdm.tqdm(valid_dataloader)
    with torch.no_grad():  #Робимо комп'ютеру легше, та не задаємо рахування градієнту для моделі
        val_loss = []
        acc = []
        for batch in valid_tqdm:
            sentence, cls = batch['vectors'], batch['target']
            sentence, cls = sentence.to(device), cls.to(device)
            
            output = model(sentence).squeeze(1)

            L = loss_fn(output, cls.float())
            val_loss.append(L.item())
            
            acc.append(metrics(output, cls).cpu())  #cpu - Перевод в процессор,щоб не скаржалось
            
            
            valid_tqdm.set_description(f"Val loss: {L.item()}")
            valid_tqdm.refresh()
            
    val_loss_g.append(np.mean(val_loss)) 
    
    if abs(val_loss_g[-2] - val_loss_g[-1]) > 0.0001: #Якщо втрати зменшуються 
        print(f"Epoch {i}, Val loss: {np.mean(val_loss)}") #Виводимо в кінці середні втрати та точність
        print(f"Epoch {i}, Accuracy: {np.mean(acc)}")
        i += 1
    else:   #інакше розриваємо тренування 
        break
    


Epoch 1


Val loss: 0.6950188875198364: 100%|██████████| 519/519 [00:05<00:00, 95.78it/s] 


Epoch 1, Val loss: 0.6912526539984466
Epoch 1, Accuracy: 0.5271281599998474
Epoch 2


Val loss: 0.6849580407142639: 100%|██████████| 519/519 [00:05<00:00, 102.79it/s]


Epoch 2, Val loss: 0.687379099500432
Epoch 2, Accuracy: 0.5525595545768738
Epoch 3


...

Val loss: 0.5368687510490417: 100%|██████████| 519/519 [00:05<00:00, 100.30it/s] 


Epoch 53, Val loss: 0.45760912329015935
Epoch 53, Accuracy: 0.7959581017494202
Epoch 54


Val loss: 0.5604839324951172: 100%|██████████| 519/519 [00:05<00:00, 97.98it/s]  


Epoch 54, Val loss: 0.4544757354810748
Epoch 54, Accuracy: 0.7985526919364929
Epoch 55


Train loss: 0.430819571018219:  14%|█▍        | 174/1211 [00:02<00:12, 86.04it/s]  